<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/Bai1/22520975_Lab2_Bai3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import os
import time
import copy
from tqdm.auto import tqdm

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- Cấu hình chung ---
DATA_DIR = '/content/drive/MyDrive/Data/VinaFood21/VinaFood21'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

if not os.path.exists(DATA_DIR):
    print(f"LỖI: Không tìm thấy thư mục '{DATA_DIR}'.")
    print("Vui lòng kiểm tra lại đường dẫn Google Drive của bạn.")

NUM_CLASSES = 21 # VinaFood21

In [ ]:
def get_dataloaders(img_size, batch_size=32):
    """Tải dữ liệu với kích thước ảnh cụ thể."""
    print(f"\nĐang tải dữ liệu với kích thước {img_size}x{img_size}...")
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    try:
        image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                          for x in ['train', 'test']}
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x=='train'), num_workers=2)
                       for x in ['train', 'test']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
        class_names = image_datasets['train'].classes
        print(f"Tải xong. {dataset_sizes['train']} ảnh train, {dataset_sizes['test']} ảnh test.")
        return dataloaders, dataset_sizes, class_names
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy thư mục 'train' hoặc 'test' trong {DATA_DIR}")
        return None, None, None

In [ ]:
def quick_evaluate(model, dataloader):
    """Hàm đánh giá nhanh (chỉ tính Acc) để xem sau mỗi epoch."""
    model.eval()
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
    acc = running_corrects.double() / total
    print(f'Test Acc (Validation): {acc:.4f}')
    model.train() # Chuyển lại chế độ train

In [ ]:
def train_model_torchvision(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện cho LeNet và ResNet (output chuẩn)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])


    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def train_model_googlenet(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện đặc biệt cho GoogLeNet (có 3 output)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                main_out, aux1_out, aux2_out = model(inputs)
                loss_main = criterion(main_out, labels)
                loss_aux1 = criterion(aux1_out, labels)
                loss_aux2 = criterion(aux2_out, labels)
                loss = loss_main + 0.3 * (loss_aux1 + loss_aux2)
                _, preds = torch.max(main_out, 1)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])

    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def evaluate_model_metrics(model, dataloader):
    """Đánh giá mô hình trên tập test và tính Precision, Recall, F1-macro."""
    print("\nĐang đánh giá chi tiết trên tập Test...")
    model = model.to(device)
    model.eval() # Chuyển sang chế độ eval

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels,
        all_preds,
        average='macro',
        zero_division=0
    )
    accuracy = accuracy_score(all_labels, all_preds)

    print("\n--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---")
    print(f'Accuracy:  {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall:    {recall:.4f}')
    print(f'F1-Score:  {f1:.4f}')
    print("-" * 39)

In [ ]:
# --- Tải Dữ Liệu ---
dataloaders_28, sizes_28, classes_28 = get_dataloaders(img_size=28, batch_size=64)
dataloaders_224, sizes_224, classes_224 = get_dataloaders(img_size=224, batch_size=32)

In [ ]:
# -------------------------------------------------------------------
# BÀI 3: ResNet-18 (224x224)
# -------------------------------------------------------------------
print("\n" + "="*50)
print("BẮT ĐẦU BÀI 3: ResNet-18")
print("="*50)

# 1. Định nghĩa BasicBlock
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    def forward(self, x):
        shortcut = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += shortcut
        out = self.relu(out)
        return out

# 2. Định nghĩa ResNet
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=21):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x); x = self.layer2(x); x = self.layer3(x); x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def ResNet18(num_classes=21):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

if dataloaders_224:
    model_resnet18 = ResNet18(num_classes=NUM_CLASSES).to(device)
    criterion_r18 = nn.CrossEntropyLoss()
    optimizer_r18 = optim.Adam(model_resnet18.parameters(), lr=0.001)

    NUM_EPOCHS_B3 = 50
    print(f"Huấn luyện ResNet-18 trong {NUM_EPOCHS_B3} epochs...")

    model_resnet18_trained = train_model_torchvision(
        model_resnet18, dataloaders_224, sizes_224,
        criterion_r18, optimizer_r18, num_epochs=NUM_EPOCHS_B3
    )

    print("\n[Bài 3: ResNet-18] Đánh giá cuối cùng:")
    evaluate_model_metrics(model_resnet18_trained, dataloaders_224['test'])
else:
    print("Bỏ qua Bài 3 do không tải được dữ liệu.")